# Resnet Implementation on Face Recognition Dataset (LFW)

The dataset is taken from this link http://vis-www.cs.umass.edu/lfw/lfw.tgz

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
print(tf.__version__)

In [3]:
physical_devices = tf.config.experimental.list_physical_devices()
physical_devices

In [4]:
tf.config.experimental.set_memory_growth(physical_devices[1], True)

## Extracting the zip file

In [5]:
import urllib.request
urllib.request.urlretrieve(' http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz', 'lfw-deepfunneled.tgz')

In [6]:
import tarfile

file = tarfile.open('./lfw-deepfunneled.tgz', 'r')
file.extractall('.')
file.close()

In [7]:
import os

base_dir = './lfw-deepfunneled' #'/content/lfw'
class_names = os.listdir(base_dir)
print(len(class_names))

## Images

In [8]:
images_dict = {}
image_path = []
for i in range(len(class_names)):
    images_folder = os.listdir(os.path.join(base_dir, class_names[i]))
    for image in images_folder:
        image_path.append(os.path.join(base_dir, class_names[i])+'/'+str(image))
        images_dict[image] = class_names[i]

In [9]:
image_path[:5]

In [10]:
col_names = ['Image', 'Label']
images_data_frame = pd.DataFrame.from_dict(images_dict, orient='index').reset_index()
images_data_frame.set_axis(col_names, axis=1, inplace=True)
images_data_frame.insert(1, "Image_path", image_path) # Adding the column Image path at position 2.
images_data_frame

In [11]:
image_dir_index = 4200 #Index of the image directory.
images = os.listdir(os.path.join(base_dir, class_names[image_dir_index]))
num_images = len(images)
plt.figure(figsize=(18, 4*(num_images/3 + 1)))
for i in range(len(images)):
    plt.subplot(int(num_images/4) + 1, 4, i+1)
    im = plt.imread(os.path.join(base_dir, class_names[image_dir_index])+'/'+str(images[i]))
#     plt.title(class_names[image_index])
    plt.imshow(im)
plt.suptitle(class_names[image_dir_index], size=18, y=0.91)

## Considering persons with greater than 50 images

In [12]:
images_data_frame.Label.value_counts()[images_data_frame.Label.value_counts() > 25]

In [13]:
persons_with_multiple_images = images_data_frame.Label.value_counts()[images_data_frame.Label.value_counts() > 25].index
len(persons_with_multiple_images)

In [14]:
import shutil
new_dir = './new'
try:
    shutil.rmtree(new_dir)
    print(f"The directory {new_dir} removed successfully")
except:
    print(f"The directory {new_dir} is not present")

try:
    os.mkdir(new_dir)
    os.mkdir(new_dir + "/train")
    os.mkdir(new_dir + "/val")
    os.mkdir(new_dir + "/test")
    print(f"The directory {new_dir} created successfully")
except OSError as error:
    print(f"Error - The directory {new_dir} is not created")

In [15]:
number_of_images = {}
for item in persons_with_multiple_images:
    source = os.path.join(base_dir, item)
    train_destination = os.path.join(new_dir + "/train", item)
    val_destination = os.path.join(new_dir + "/val", item)
    test_destination = os.path.join(new_dir + "/test", item)
    try:
        os.mkdir(train_destination)
        os.mkdir(val_destination)
        os.mkdir(test_destination)
    except OSError as error: 
        pass
    i=0
    for image_item in os.listdir(source):
        source_image = os.path.join(source, image_item)
        if i<20:
            shutil.copy(source_image, train_destination)
        elif i<25:
            shutil.copy(source_image, val_destination)
        else:
            shutil.copy(source_image, test_destination)
        i += 1
        number_of_images[item] = [len(os.listdir(train_destination)), len(os.listdir(val_destination)), len(os.listdir(test_destination))]
    # print(image_item, source_image)

In [16]:
col_names = ['Person', 'Train Images', 'Validation Images', 'Test Images']
df = pd.DataFrame.from_dict(number_of_images, orient='index').reset_index()
df.set_axis(col_names, axis=1, inplace=True)
df

### Image Data Generator

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
# All images will be rescaled by 1./255.
datagen = ImageDataGenerator(rescale=1./255, 
                             #rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
                             zoom_range = 0.1, # Randomly zoom image 
                             width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                             height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                             shear_range=0.1,
                             vertical_flip=False,
                             horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
train_generator = datagen.flow_from_directory(new_dir + "/train",
                                              target_size = (250 ,250),
                                              batch_size = 32,
                                              class_mode = 'categorical')

validation_generator = val_datagen.flow_from_directory(new_dir + "/val",
                                                       target_size = (250 ,250),
                                                       batch_size = 32,
                                                       class_mode = 'categorical')

## RESNET 50

In [20]:
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model

In [21]:
def my_resnet(pretrained):
    # We do not want the fully-connected layer at the top of network. 
    resnet = ResNet50(include_top=False, input_shape=(250, 250, 3))

    output = resnet.layers[-1].output
    # output = keras.layers.MaxPooling2D(pool_size=(2,2))(output)
    # output = keras.layers.Flatten()(output)

    resnet = Model(resnet.input, outputs=output)

    # We freeze the weights of the model by setting trainable as "False"
    for layer in resnet.layers:
        layer.trainable = not pretrained
        
    return resnet

    # resnet.summary()

### Adding more layers(trainable layers) on top of the layers of resnet model

In [22]:
len(persons_with_multiple_images)

In [23]:
from keras.models import Sequential

def my_resnet_model(pretrained=False):
    model = Sequential([
            keras.layers.InputLayer(input_shape=(250, 250, 3)),
            my_resnet(pretrained),
            keras.layers.BatchNormalization(),
            keras.layers.Conv2D(4*1024, (3,3), padding='valid', activation='relu'),
#             keras.layers.MaxPooling2D(pool_size=(2,2)),
            keras.layers.BatchNormalization(),
#             keras.layers.Conv2D(4*1024, (3,3), padding='valid', activation='relu'),
#             keras.layers.MaxPooling2D(pool_size=(2,2)),
#             keras.layers.BatchNormalization(),
#             keras.layers.Conv2D(8*1024, (3,3), padding='valid', activation='relu'),
#             keras.layers.MaxPooling2D(pool_size=(2,2)),
#             keras.layers.Conv2D(1024, (3,3), padding='valid', activation='relu'),
#             keras.layers.MaxPooling2D(pool_size=(2,2)),
            keras.layers.Flatten(),
#             keras.layers.Dense(16348, activation='relu', name='dense_1'),
#             keras.layers.Dropout(0.2),
            keras.layers.Dense(4*1024, activation='relu', name='dense_1'),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(1024, activation='relu', name='dense_2'),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(256, activation='relu', name='dense_3'),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(len(persons_with_multiple_images), activation='softmax', name='dense_final')
    ])
    
    print(model.summary())
    
    model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=1e-5),
              metrics=['accuracy'])
    
    return model
    
def my_simple_model():
    model = Sequential()
    model.add(keras.layers.Conv2D(32, (3, 3), input_shape = (250, 250, 3), activation = 'relu'))
    model.add(keras.layers.MaxPooling2D(pool_size = (2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation = 'relu'))
    model.add(keras.layers.MaxPooling2D(pool_size = (2, 2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units = 1024, activation = 'relu'))
    model.add(keras.layers.Dense(units = 128, activation = 'relu'))
    
    # as we are training on several classes, we need several classification units (one for each class). We also use a softmax activation function
    model.add(keras.layers.Dense(units = len(persons_with_multiple_images), activation = 'softmax'))
    
    print(model.summary())
    
    model.compile(loss=keras.losses.CategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(learning_rate=1e-5),
              metrics=['accuracy'])
    
    return model

### Plots

In [24]:
def plots(history):
    plt.figure(figsize=(18,10))
    
    plt.subplot(2,1,1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train accuracy', 'test accuracy'], loc='upper left')
    
    plt.subplot(2,1,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train loss', 'test loss'], loc='upper right')

    plt.show()

## Making different Models

### 1. Simple CNN

In [ ]:
model = my_simple_model()

In [ ]:
history = model.fit(train_generator, epochs=100, validation_data = validation_generator)

In [ ]:
plots(history)

In [ ]:
del model
tf.keras.backend.clear_session()

### 2. Resnet with pretrained weights

In [25]:
model = my_resnet_model(pretrained=True)

In [26]:
history = model.fit(train_generator, epochs=100, validation_data = validation_generator)

In [28]:
plots(history)

In [ ]:
del model
tf.keras.backend.clear_session()

In [ ]:
tf.keras.backend.clear_session()

### 3. Resnet without pretrained weihts

In [25]:
model = my_resnet_model(pretrained=False)

In [26]:
history = model.fit(train_generator, epochs=100, validation_data = validation_generator)

In [28]:
plots(history)

## Test on Images

In [29]:
test_directory = new_dir + "/test"
persons = os.listdir(test_directory)

persons_dict = {}
for i in range(len(persons)):
    images_folder = os.listdir(os.path.join(test_directory, persons[i]))
    for image in images_folder:
        person_image_path = os.path.join(test_directory, persons[i]+'/'+str(image))
        persons_dict[image] = [person_image_path, persons[i]]
        
col_names = ['Image', 'Image_path', 'Label']
persons_data_frame = pd.DataFrame.from_dict(persons_dict, orient='index').reset_index()
persons_data_frame.set_axis(col_names, axis=1, inplace=True)
persons_data_frame

In [30]:
names_dict = {v:k for k, v in train_generator.class_indices.items()}
names_dict

In [53]:
Image_numbers = [800, 200, 1200]

for Image_number in Image_numbers:
    plt.figure()
    im = plt.imread(persons_data_frame.Image_path[Image_number])
    input_shape = im.shape
    img = im.reshape(-1, input_shape[0], input_shape[1], input_shape[2]).astype('float32') / 255.0
    prediction = model.predict(img)
    result = names_dict[np.argmax(prediction)]
    format_float = "{:.4f}".format(np.max(prediction)*100)
    plt.title("The image is of: " + str(persons_data_frame.Label[Image_number]))
    plt.suptitle(f"The image predicted is of {result} with {format_float}% probability", size=18, y=0.5, x=1.5)
    plt.imshow(im)

In [46]:
cnt = 0
total = 0
for index, row in persons_data_frame[['Image_path', 'Label']].iterrows():
    im = plt.imread(row['Image_path'])
    input_shape = im.shape
    img = im.reshape(-1, input_shape[0], input_shape[1], input_shape[2]).astype('float32') / 255.0
    result = names_dict[np.argmax(model.predict(img))]
    if(result==row['Label']):
        cnt += 1
    total += 1
print(f"{cnt} number of images are correctly recognized out of {total} images")
print(f"The accuracy is {cnt/total}")

In [ ]:
del model

In [ ]:
tf.keras.backend.clear_session()